In [2]:
import http.client

conn = http.client.HTTPSConnection("free-api-live-football-data.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "f43b73513cmsh693e1cd4c0cfae5p19d43cjsn4363c8b097f6",
    'x-rapidapi-host': "free-api-live-football-data.p.rapidapi.com"
}

conn.request("GET", "/football-get-all-leagues-with-countries", headers=headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

{"status":"success","response":{"leagues":[{"ccode":"ALB","name":"Albania","leagues":[{"id":10175,"name":"Super Cup","localizedName":"Super Cup","logo":"https://images.fotmob.com/image_resources/logo/leaguelogo/dark/10175.png"},{"id":260,"name":"Superiore","localizedName":"Superiore","logo":"https://images.fotmob.com/image_resources/logo/leaguelogo/dark/260.png"},{"id":9173,"name":"Superiore Qualification","localizedName":"Superiore Qualification","logo":"https://images.fotmob.com/image_resources/logo/leaguelogo/dark/9173.png"}],"localizedName":"Albania"},{"ccode":"ALG","name":"Algeria","leagues":[{"id":516,"name":"Ligue 1","localizedName":"Ligue 1","logo":"https://images.fotmob.com/image_resources/logo/leaguelogo/dark/516.png"}],"localizedName":"Algeria"},{"ccode":"ARG","name":"Argentina","leagues":[{"id":112,"name":"Liga Profesional","localizedName":"Liga Profesional","logo":"https://images.fotmob.com/image_resources/logo/leaguelogo/dark/112.png"},{"id":8965,"name":"Primera B Naciona

In [3]:
import http.client

conn = http.client.HTTPSConnection("free-api-live-football-data.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "f43b73513cmsh693e1cd4c0cfae5p19d43cjsn4363c8b097f6",
    'x-rapidapi-host': "free-api-live-football-data.p.rapidapi.com"
}

conn.request("GET", "/football-get-list-all-team?leagueid=53", headers=headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))


{"status":"success","response":{"list":[{"name":"Paris Saint-Germain","shortName":"PSG","id":9847,"deduction":null,"ongoing":null,"played":13,"wins":10,"draws":3,"losses":0,"scoresStr":"37-11","goalConDiff":26,"pts":33,"idx":1,"qualColor":"#2AD572","logo":"https://images.fotmob.com/image_resources/logo/teamlogo/9847_large.png"},{"name":"Marseille","shortName":"Marseille","id":8592,"deduction":null,"ongoing":null,"played":13,"wins":8,"draws":2,"losses":3,"scoresStr":"29-17","goalConDiff":12,"pts":26,"idx":2,"qualColor":"#2AD572","logo":"https://images.fotmob.com/image_resources/logo/teamlogo/8592_large.png"},{"name":"Monaco","shortName":"Monaco","id":9829,"deduction":null,"ongoing":null,"played":13,"wins":8,"draws":2,"losses":3,"scoresStr":"22-12","goalConDiff":10,"pts":26,"idx":3,"qualColor":"#2AD572","logo":"https://images.fotmob.com/image_resources/logo/teamlogo/9829_large.png"},{"name":"Lille","shortName":"Lille","id":8639,"deduction":null,"ongoing":null,"played":13,"wins":6,"draws"

In [4]:
import http.client

conn = http.client.HTTPSConnection("free-api-live-football-data.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "f43b73513cmsh693e1cd4c0cfae5p19d43cjsn4363c8b097f6",
    'x-rapidapi-host': "free-api-live-football-data.p.rapidapi.com"
}

conn.request("GET", "/football-get-standing-all?leagueid=53", headers=headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

{"status":"success","response":{"standing":[{"name":"Paris Saint-Germain","shortName":"PSG","id":9847,"pageUrl":"/teams/9847/overview/paris-saint-germain","deduction":null,"ongoing":null,"played":13,"wins":10,"draws":3,"losses":0,"scoresStr":"37-11","goalConDiff":26,"pts":33,"idx":1,"qualColor":"#2AD572"},{"name":"Marseille","shortName":"Marseille","id":8592,"pageUrl":"/teams/8592/overview/marseille","deduction":null,"ongoing":null,"played":13,"wins":8,"draws":2,"losses":3,"scoresStr":"29-17","goalConDiff":12,"pts":26,"idx":2,"qualColor":"#2AD572"},{"name":"Monaco","shortName":"Monaco","id":9829,"pageUrl":"/teams/9829/overview/monaco","deduction":null,"ongoing":null,"played":13,"wins":8,"draws":2,"losses":3,"scoresStr":"22-12","goalConDiff":10,"pts":26,"idx":3,"qualColor":"#2AD572"},{"name":"Lille","shortName":"Lille","id":8639,"pageUrl":"/teams/8639/overview/lille","deduction":null,"ongoing":null,"played":13,"wins":6,"draws":5,"losses":2,"scoresStr":"21-13","goalConDiff":8,"pts":23,"i

In [5]:
import http.client

conn = http.client.HTTPSConnection("free-api-live-football-data.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "f43b73513cmsh693e1cd4c0cfae5p19d43cjsn4363c8b097f6",
    'x-rapidapi-host': "free-api-live-football-data.p.rapidapi.com"
}

conn.request("GET", "/football-league-team?teamid=9748", headers=headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

{"status":"success","response":{"details":{"id":9748,"type":"team","name":"Lyon","latestSeason":"2024/2025","shortName":"Lyon","country":"FRA","faqJSONLD":{"@context":"https://schema.org","@type":"FAQPage","mainEntity":[{"@type":"Question","name":"When is Lyon's next match?","acceptedAnswer":{"@type":"Answer","text":"Lyon's next match is at 20:00 GMT on Sat, 7 Dec 2024 against Angers."}},{"@type":"Question","name":"Who is Lyon's top scorer?","acceptedAnswer":{"@type":"Answer","text":"Alexandre Lacazette has scored the most goals for Lyon, with 5 goals."}},{"@type":"Question","name":"Who is Lyon's best player?","acceptedAnswer":{"@type":"Answer","text":"Corentin Tolisso is the top-rated player for Lyon with a FotMob rating of 7.39."}},{"@type":"Question","name":"Who has the most assists for Lyon?","acceptedAnswer":{"@type":"Answer","text":"Clinton Mata has the most assists on Lyon, with 3 assists."}},{"@type":"Question","name":"Where is Lyon's stadium?","acceptedAnswer":{"@type":"Answer

In [6]:
import http.client

conn = http.client.HTTPSConnection("free-api-live-football-data.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "f43b73513cmsh693e1cd4c0cfae5p19d43cjsn4363c8b097f6",
    'x-rapidapi-host': "free-api-live-football-data.p.rapidapi.com"
}

conn.request("GET", "/football-get-all-matches-by-league?leagueid=53", headers=headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

{"status":"success","response":{"matches":[{"id":"4513014","pageUrl":"/matches/le-havre-vs-paris-saint-germain/36af6s#4513014","opponent":{"id":"9746","name":"Le Havre","score":1},"home":{"id":"9746","name":"Le Havre","score":1},"away":{"id":"9847","name":"PSG","score":4},"displayTournament":true,"notStarted":false,"tournament":{},"status":{"utcTime":"2024-08-16T18:45:00Z","finished":true,"started":true,"cancelled":false,"awarded":false,"scoreStr":"1 - 4","reason":{"short":"FT","shortKey":"fulltime_short","long":"Full-Time","longKey":"finished"}}},{"id":"4513007","pageUrl":"/matches/brest-vs-marseille/2f6tbp#4513007","opponent":{"id":"8521","name":"Brest","score":1},"home":{"id":"8521","name":"Brest","score":1},"away":{"id":"8592","name":"Marseille","score":5},"displayTournament":true,"notStarted":false,"tournament":{},"status":{"utcTime":"2024-08-17T15:00:00Z","finished":true,"started":true,"cancelled":false,"awarded":false,"scoreStr":"1 - 5","reason":{"short":"FT","shortKey":"fulltim

In [12]:
import http.client

conn = http.client.HTTPSConnection("free-api-live-football-data.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "f43b73513cmsh693e1cd4c0cfae5p19d43cjsn4363c8b097f6",
    'x-rapidapi-host': "free-api-live-football-data.p.rapidapi.com"
}

conn.request("GET", "/football-get-matches-by-date-and-league?date=20241101", headers=headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

{"status":"success","response":[{"ccode":"ITA","id":55,"primaryId":55,"name":"Serie A","matches":[{"id":4535422,"leagueId":55,"time":"31.10.2024 20:45","home":{"id":10171,"score":1,"name":"Como","redCards":1,"longName":"Como"},"away":{"id":8543,"score":5,"name":"Lazio","redCards":1,"longName":"Lazio"},"eliminatedTeamId":null,"statusId":6,"tournamentStage":"10","status":{"utcTime":"2024-10-31T19:45:00.000Z","numberOfHomeRedCards":1,"numberOfAwayRedCards":1,"halfs":{"firstHalfStarted":"31.10.2024 20:47:10","secondHalfStarted":"31.10.2024 21:50:19"},"finished":true,"started":true,"cancelled":false,"awarded":false,"scoreStr":"1 - 5","reason":{"short":"FT","shortKey":"fulltime_short","long":"Full-Time","longKey":"finished"}},"timeTS":1730403900000},{"id":4535428,"leagueId":55,"time":"31.10.2024 20:45","home":{"id":8686,"score":1,"name":"Roma","longName":"Roma"},"away":{"id":9804,"score":0,"name":"Torino","longName":"Torino"},"eliminatedTeamId":null,"statusId":6,"tournamentStage":"10","statu